What i want to do: 
- read in postcode dataset
- loop through the code point files 
- join postcodes in each file 
- make sure these are saved and each time loop through a file it doesn't delete earlier merges. 

In [1]:
# Set libraries
import pandas as pd
import glob
import os
import numpy as np
import datetime
import string
import re

In [2]:
# Read in postcode dataset
SWW_postcode = pd.read_csv(r'/Users/annalisasheehan/Documents/Imperial-Diabetes/postcodes/SWW postcodes.csv')

In [11]:
dtypeCount =[SWW_postcode.iloc[:,i].apply(type).value_counts() for i in range(SWW_postcode.shape[1])]

In [12]:
dtypeCount

[<class 'str'>    11011
 Name: date, dtype: int64, <class 'str'>    11011
 Name: postcode, dtype: int64]

In [10]:
# Change the postcode field to string
SWW_postcode['postcode']= SWW_postcode['postcode'].astype('str')

In [25]:
mask = (SWW_postcode['postcode'].str.len()>7)
mask2 = (SWW_postcode['postcode'].str.len()<8)

In [18]:
df1 = SWW_postcode.loc[mask]

In [26]:
df2 = SWW_postcode.loc[mask2]

In [ ]:
df2

In [ ]:
SWW_postcode

In [39]:
df1['First']= df1['postcode'].str[:3]

/Users/annalisasheehan/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [33]:
df1.loc[:,('postcode')] = df1.loc[:,('postcode')].map(lambda x: x.lstrip(' '))

/Users/annalisasheehan/anaconda/lib/python3.5/site-packages/pandas/core/indexing.py:545: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item_labels[indexer[info_axis]]] = value


In [31]:
df1['postcode'] = df1['postcode'].map(lambda x: x.lstrip(' '))



/Users/annalisasheehan/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [ ]:
SWW_postcode = SWW_postcode.set_index('postcode')

In [ ]:
SWW_postcode

In [ ]:
files = glob.glob(r'/Users/annalisasheehan/Documents/Imperial-Diabetes/postcodes/codepo_gb/Data/CSV/*')

In [ ]:
files

In [ ]:
labels = pd.read_csv(r'/Users/annalisasheehan/Documents/Imperial-Diabetes/postcodes/codepo_gb/Doc/Code-Point_Open_Column_Headers.csv', header = None)

In [ ]:
labels

In [ ]:
labels = labels.set_index([0])

In [ ]:
labels

for file in files:
    postcode = pd.read_csv(r'{fname}'.format(fname=file), header=None)
    postcode = postcode.set_index([0])
    data = pd.concat([labels,postcode])
    data.columns = data.iloc[0]
    data = data.reindex(data.index.drop('Postcode'))
    SWW_postcode.merge(data, how='left', left_index=True, right_index=True)
    
    
    print('Processed {fname}'.format(fname=file))

In [ ]:
postcode = pd.read_csv(r'/Users/annalisasheehan/Documents/Imperial-Diabetes/postcodes/codepo_gb/Data/CSV/ex.csv', header = None)

In [ ]:
postcode

In [ ]:
postcode = postcode.set_index([0])

In [ ]:
data = pd.concat([labels,postcode])

In [ ]:
data

In [ ]:
data.columns = data.iloc[0]

In [ ]:
data = data.reindex(data.index.drop('Postcode'))

In [ ]:
data

In [ ]:
SWW_postcode = SWW_postcode.merge(data, how='left', left_index=True, right_index=True)

In [ ]:
SWW_postcode

In [ ]:
data

In [ ]:
data.insert(0, 'date', 'NaN')

In [ ]:
data['date']= pd.to_datetime(data['date'])

In [ ]:
SWW_postcode[SWW_postcode.isnull()] = data

In [ ]:
SWW_postcode.to_csv(r'/Users/annalisasheehan/Documents/Imperial-Diabetes/postcodes/test.csv')